In [32]:
# Import Dependencies
import json
import requests
from config import api_key
from config import gkey
from pprint import pprint
import pandas as pd
from datetime import date
from datetime import timedelta
import requests
import urllib
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import geopy.distance as gp
import warnings
warnings.filterwarnings('ignore')

# Food Deserts

A Food Desert is an area that has limited access to affordable and nutritious food. Food deserts tend to be inhabited by low-income residents with reduced mobility; this makes them a less attractive market for large supermarket chains. Food deserts lack suppliers of fresh foods, such as meats, fruits, and vegetables. Instead, the available foods are often processed and high in sugar and fats, which are known contributors to the proliferation of obesity in the U.S.

### Beginning the data-gathering process

We found that the Percentage of Free or Reduced Lunches (Percent_FRL) at school is a strong indicator of poverty in an area:
https://nces.ed.gov/blogs/nces/post/free-or-reduced-price-lunch-a-proxy-for-poverty

Using this metric, we identified 90 schools across 7 counties of Georgia with Percent_FRL of >= 92%. We collected these school names, physical addresses and the their Percent_FRL in a CSV file.

### Approach
1. Get the name and addresses of all the schools which have the >90% FRL (free-reduced lunch) (%FRL is used as an indicator of poverty).
2. Find out the number of grocery stores within 5-10 miles of these schools (usually people live near the schools and higher percentage FRL means that the neighborhood is poor).
3. Create a Heat Map these grocery stores.
4. Find out any Fast Food joints / Dollar Stores within 5 miles of these locations. Usually in Food Deserts there is proliferation of these establishments selling processed foods.
5. Creat a Heat Map these Fast Food places / Dollar Stores.

In [61]:
infile = pd.read_csv("../Datasets/GA_Counties_Schools.csv")
school_df = pd.DataFrame(infile)
school_df.head()

,County_ID,County_Name,School_ID,School_Name,Percent_FRL,Address,City,State,ZipCode
0,605,Baldwin County,189,Baldwin High School,92.86,155 Ga Highway 49 W,Milledgeville,GA,31061
1,605,Baldwin County,300,Lakeview Academy,92.78,220 N Abc Dr,Milledgeville,GA,31061
2,605,Baldwin County,302,Lakeview Primary,92.92,372 Blandy Rd NW,Milledgeville,GA,31061
3,605,Baldwin County,303,Midway Hills Academy,92.90,101 Carl Vinson Rd SE,Milledgeville,GA,31061
4,605,Baldwin County,301,Midway Hills Primary,92.86,375 Blandy Rd NW,Milledgeville,GA,31061


## Getting Coordinates of the Schools

In [59]:
school_coordf = pd.DataFrame(columns=['School_ID', 'County_Name', 'School_Name', 'Latitude', 'Longitude'])
                                   
errors = []

for row in school_df.itertuples():
    County_Name = row.County_Name
    School_ID = row.School_ID
    School_Name = row.School_Name
    Zip = str(row.ZipCode)
    school_addr = row.Address+','+row.City+','+row.State+','+Zip
    api_response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'.format(school_addr, gkey))
    api_response_dict = api_response.json()

    try:
        if api_response_dict['status'] == 'OK':
            Latitude = api_response_dict['results'][0]['geometry']['location']['lat']
            Longitude = api_response_dict['results'][0]['geometry']['location']['lng']

            current_record = [School_ID, County_Name, School_Name, Latitude, Longitude]
            school_coordf.loc[len(school_coordf)] = current_record
   
    except KeyError:
            errors.append('Key Error')

school_coordf



,School_ID,County_Name,School_Name,Latitude,Longitude
0,189,Baldwin County,Baldwin High School,33.072068,-83.258449
1,300,Baldwin County,Lakeview Academy,33.090509,-83.264271
2,302,Baldwin County,Lakeview Primary,33.080177,-83.268580
3,303,Baldwin County,Midway Hills Academy,33.019280,-83.227823
4,301,Baldwin County,Midway Hills Primary,33.079949,-83.271429
...,...,...,...,...,...
111,205,Wilkes County,Washington-Wilkes Elementary School,33.734583,-82.725189
112,105,Wilkes County,Washington-Wilkes Primary School,33.728877,-82.720109
113,105,Wilkes County,Washington-Wilkes Primary School,33.728877,-82.720109
114,210,Wilkes County,Washington-Wilkes Middle School,33.760145,-82.732498


In [62]:
school_df = school_df.merge(school_coordf[['School_ID','Latitude', 'Longitude']], on = 'School_ID', how = 'left')
school_df

,County_ID,County_Name,School_ID,School_Name,Percent_FRL,Address,City,State,ZipCode,Latitude,Longitude
0,605,Baldwin County,189,Baldwin High School,92.86,155 Ga Highway 49 W,Milledgeville,GA,31061,33.072068,-83.258449
1,605,Baldwin County,300,Lakeview Academy,92.78,220 N Abc Dr,Milledgeville,GA,31061,33.090509,-83.264271
2,605,Baldwin County,302,Lakeview Primary,92.92,372 Blandy Rd NW,Milledgeville,GA,31061,33.080177,-83.268580
3,605,Baldwin County,303,Midway Hills Academy,92.90,101 Carl Vinson Rd SE,Milledgeville,GA,31061,33.019280,-83.227823
4,605,Baldwin County,301,Midway Hills Primary,92.86,375 Blandy Rd NW,Milledgeville,GA,31061,33.079949,-83.271429
...,...,...,...,...,...,...,...,...,...,...,...
163,757,Wilkes County,105,Washington-Wilkes Primary School,94.16,910 E Robert Toombs Ave,Washington,GA,30673,33.593735,-84.318128
164,757,Wilkes County,105,Washington-Wilkes Primary School,94.16,910 E Robert Toombs Ave,Washington,GA,30673,33.728877,-82.720109
165,757,Wilkes County,105,Washington-Wilkes Primary School,94.16,910 E Robert Toombs Ave,Washington,GA,30673,33.728877,-82.720109
166,757,Wilkes County,210,Washington-Wilkes Middle School,94.22,1180 Tignall Rd,Washington,GA,30673,33.760145,-82.732498


In [84]:
stores = []
for row in school_df.itertuples():
    
    County_Name = row.County_Name
    School_ID = row.School_ID
    School_Name = row.School_Name
      
    School_Lat = row.Latitude
    School_Long = row.Longitude 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"key": gkey, 
          "location": f"{School_Lat},{School_Long}", 
          "radius": 10000, 
          "type": "supermarket"}

    data = requests.get(base_url, params = params).json()
    
    try:
        if data['status'] == 'OK':
            i = 0
            for row in data:
                Store_Lat = data['results'][i]['geometry']['location']['lat']
                Store_Long = data['results'][i]['geometry']['location']['lng']
                Store_Name = data['results'][i]['name']
                Store_Address = data['results'][i]['vicinity']
                Store_PlaceID = data['results'][i]['place_id']
                
                current_record = [School_ID, County_Name, School_Name, School_Lat, School_Long,\
                                    Store_Name, Store_Lat, Store_Long, Store_Address, Store_PlaceID]
                stores.append(current_record)
                i = i + 1
                
    except IndexError:
        errors.append('Index Out of Range')    
    


store_df = pd.DataFrame(stores, columns=['School_ID', 'County_Name', 'School_Name', 'School_Lat', 'School_Long', 'Store_Name',\
            'Store_Lat','Store_Long', 'Store_Address','Store_PlaceID'])
store_df




,School_ID,County_Name,School_Name,School_Lat,School_Long,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,189,Baldwin County,Baldwin High School,33.072068,-83.258449,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
1,189,Baldwin County,Baldwin High School,33.072068,-83.258449,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
2,189,Baldwin County,Baldwin High School,33.072068,-83.258449,Family Dollar,33.055031,-83.239311,"355 Allen Memorial Drive, Milledgeville",ChIJpUDdw1vJ9ogReo5URShyfMA
3,300,Baldwin County,Lakeview Academy,33.090509,-83.264271,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
4,300,Baldwin County,Lakeview Academy,33.090509,-83.264271,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
...,...,...,...,...,...,...,...,...,...,...
550,210,Wilkes County,Washington-Wilkes Middle School,33.760145,-82.732498,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM
551,210,Wilkes County,Washington-Wilkes Middle School,33.760145,-82.732498,Family Dollar,33.725677,-82.718119,"164 Bi-Lo Plaza, Washington",ChIJlYwf8G2l94gRxbbIZSoRgvw
552,110,Wilkes County,Washington-Wilkes Comprehensive High School,33.760145,-82.732498,Terry's IGA,33.736149,-82.739447,"115 North Spring Street, Washington",ChIJidCGaIuv94gR-pjx2WR5QAY
553,110,Wilkes County,Washington-Wilkes Comprehensive High School,33.760145,-82.732498,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM
